# Webscrapping indeed using scrapfly
To start scrapping from indeed using scrapfly you login to https://scrapfly.io/ and register. From scrapfly copy the API key that you are going to use below.
install the scrapfly library through the following command
pip install scrapfly-sdk

In [ ]:
import re
import json
import csv
from scrapfly import ScrapflyClient, ScrapeConfig
from bs4 import BeautifulSoup

# Initialize Scrapfly with your API key
scrapfly = ScrapflyClient(key="Replace with your API key") #API key from scrapfly

role = input("Enter the role you are searching for:").strip() # Enter the job you are searching for
location = input("Enter your prefered working location:").strip() # Enter your prefered location such as Remote or your current location

def parse_search_page(html: str):
    # Extract JSON-like data for job listings from HTML
    data_match = re.findall(r'window.mosaic.providerData\["mosaic-provider-jobcards"\]=(\{.+?\});', html)
    
    if not data_match:
        print("No data found on page.")
        return None

    data = json.loads(data_match[0])
    job_cards = data["metaData"]["mosaicProviderJobCardsModel"]["results"]
    print(job_cards)
    # Parsing specific job details
    jobs = []
    for job in job_cards:
        # Use BeautifulSoup to remove any HTML tags in the description
        description_html = job.get("snippet", "")
        description = BeautifulSoup(description_html, "html.parser").get_text()

        job_info = {
            "title": job.get("title"),
            "company": job.get("company"),
            "location": job.get("formattedLocation") or "Not specified",
            "description": description,
            "rating": job.get("companyRating"),
            "review_count" : job.get("companyReviewCount") or "Not reviewed",
            "posted_date": job.get("formattedRelativeTime"),
            "job_link": "https://www.indeed.com" + job.get("link") if job.get("link") else None,
            "link" : job.get("thirdPartyApplyUrl"),
            "salary": job.get("salarySnippet", {}).get("text", "Not stated")  
        }
        jobs.append(job_info)
    
    return jobs

# Scraping configuration and execution
result = scrapfly.scrape(
    ScrapeConfig(
        url="https://www.indeed.com/jobs?q=" + role +"&l=" + location,
        asp=True,
    )
)

# Parsing results
parsed_jobs = parse_search_page(result.content)

# Saving to CSV file
if parsed_jobs:
    with open('indeed_jobs.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["title", "company", "location", "description", "rating", "review_count",  "posted_date", "job_link", "link", "salary"])
        writer.writeheader()
        writer.writerows(parsed_jobs)

    print("Data saved to indeed_jobs.csv")
else:
    print("No jobs data to save.")


Enter the role you are searching for: data analyst
Enter your prefered working location: remote


[{'appliedOrGreater': False, 'blobKey': 'SoCV67M364qqYDygAZ0LbzkdCdPP', 'clickLoggingUrl': '/rc/clk?jk=ceb5d3f4de8c61be&from=jasx&tk=1iboh9dlumv8q882&bb=xvoWqzpJBNZ7NRdcMGPxyizrMAI7RVRiUXftulfoHClsyfhkFIcEqbf3KOnalEu6Q0tClFtEZtHX-iC6saZzJXqOvvOWBrdpdYcNVhTxxTlLtQ3pPBJ77ULVyau-R5bG&xkcb=SoCV67M364qqYDygAZ0LbzkdCdPP&vjs=2', 'company': 'Harbor Compliance', 'companyBrandingAttributes': {'logoUrl': 'https://d2q79iu7y748jz.cloudfront.net/s/_squarelogo/256x256/8656336ee0615c6f3cc1e5be699dcf8b', 'shownForBrandedJobPackage': False}, 'companyIdEncrypted': '1224185b2123a6b9', 'companyOverviewLink': '/cmp/Harbor-Compliance', 'companyOverviewLinkCampaignId': '', 'companyRating': 3.6, 'companyReviewCount': 16, 'companyReviewLink': '/cmp/Harbor-Compliance/reviews', 'companyReviewLinkCampaignId': 'cmplinktst2', 'createDate': 1729112280000, 'd2iEnabled': False, 'displayTitle': 'Research Data Analyst', 'dradisJob': False, 'employerAssistEnabled': False, 'employerResponsive': False, 'encryptedFccompanyId

In [109]:
import pandas as pd

df = pd.read_csv("indeed_jobs.csv")
df

,title,company,location,description,rating,review_count,posted_date,job_link,link,salary
0,Research Data Analyst,Harbor Compliance,Remote,\nPerform ongoing data due diligence through m...,3.6,16,17 days ago,https://www.indeed.com/rc/clk?jk=ceb5d3f4de8c6...,http://www.indeed.com//applystart?jk=ceb5d3f4d...,Not stated
1,Senior Data Analyst,"Dispatch, LLC",Remote,\nStrategic Leadership in Business Intelligenc...,3.6,34,1 day ago,https://www.indeed.com/rc/clk?jk=275ee6d036f9e...,http://www.indeed.com//applystart?jk=275ee6d03...,Not stated
2,Senior Data Analyst,Ascension,Remote,\nConduct complex quality data analysis and cl...,3.6,8919,1 day ago,https://www.indeed.com/rc/clk?jk=a6b671ba6e006...,http://www.indeed.com//applystart?jk=a6b671ba6...,Not stated
3,Cybersecurity Data Analyst,Hunter Strategy,Remote,"\nExtract data from various databases, perform...",0.0,Not reviewed,30 days ago,https://www.indeed.com/rc/clk?jk=92441bd8f910d...,http://www.indeed.com//applystart?jk=92441bd8f...,Not stated
4,Senior Data Analyst - Remote,East West,"Avon, CO",\nCollaborate with team members to integrate d...,4.1,16,3 days ago,https://www.indeed.com/rc/clk?jk=e22b4fb42f367...,http://www.indeed.com//applystart?jk=e22b4fb42...,"$85,000 - $115,000 a year"
5,"Data Analyst IV, Web",AgelessRx,Remote,\nConduct quality checks and drive continuous ...,0.0,Not reviewed,30+ days ago,https://www.indeed.com/rc/clk?jk=7a3f6f1fdd826...,http://www.indeed.com//applystart?jk=7a3f6f1fd...,"$120,000 - $130,000 a year"
6,Data Analyst,Jackson Lewis,"Denver, CO",\nAbility to interpret and report on client da...,3.3,132,4 days ago,https://www.indeed.com/rc/clk?jk=f08fff9d6a5ed...,http://www.indeed.com//applystart?jk=f08fff9d6...,$29.00 - $37.40 an hour
7,Senior Data Analyst,Nike,"Beaverton, OR",\nMust have a Master’s degree in Computer Info...,4.1,12699,2 days ago,https://www.indeed.com/rc/clk?jk=c8d98d5a21302...,http://www.indeed.com//applystart?jk=c8d98d5a2...,Not stated
8,Senior Data Analyst,Nike,"Beaverton, OR",\nMust have a Master’s degree in Computer Info...,4.1,12699,2 days ago,https://www.indeed.com/rc/clk?jk=e62262bec5b28...,http://www.indeed.com//applystart?jk=e62262bec...,Not stated
9,Continuous Diagnostic and Mitigation (CDM) Dat...,"Valiant Solutions, LLC",Remote,"\nPerform data analysis, data modeling and dev...",3.5,14,11 days ago,https://www.indeed.com/rc/clk?jk=e751c442720cc...,http://www.indeed.com//applystart?jk=e751c4427...,"$124,500 - $138,100 a year"


In [120]:
from IPython.display import FileLink

# Display download link for indeed_jobs.csv to your local computer
FileLink(r'indeed_jobs.csv')

C:\Users\Administrator\indeed_jobs.csv

In [11]:
df.columns

Index(['title', 'company', 'location', 'description', 'posted_date',
       'job_link', 'salary'],
      dtype='object')

In [19]:
df['description']

0     \nConduct complex quality data analysis and cl...
1     \nStrategic Leadership in Business Intelligenc...
2     \nMust have a Master’s degree in Computer Info...
3     \nDevelop and enforce data governance policies...
4     \nCollaborate with team members to integrate d...
5     \nRetrieve data from customer systems, includi...
6     \nWork directly with REDCap data technicians a...
7     \nContinuously learn new tools and systems to ...
8     \nExtract data from various databases, perform...
9     \nReporting to the Software Engineering Manage...
10    \nConduct quality checks and drive continuous ...
11    \nReporting to the Software Engineering Manage...
12    \nPerform ongoing data due diligence through m...
13    \nPerform data analysis, data modeling and dev...
14    \nScripting and Data Proficiency: Skilled in d...
Name: description, dtype: object

In [63]:
df.duplicated().sum()

0

In [ ]:
import re
import json
import csv
from scrapfly import ScrapflyClient, ScrapeConfig
from bs4 import BeautifulSoup

# Initialize Scrapfly with your API key
scrapfly = ScrapflyClient(key="Replace with your API key") #API key from scrapfly

role = input("Enter the role you are searching for:").strip() # Enter the job you are searching for
location = input("Enter your prefered working location:").strip() # Enter your prefered location such as Remote or your current location

def parse_search_page(html: str):
    # Extract JSON-like data for job listings from HTML
    data_match = re.findall(r'window.mosaic.providerData\["mosaic-provider-jobcards"\]=(\{.+?\});', html)
    
    if not data_match:
        print("No data found on page.")
        return None

    data = json.loads(data_match[0])
    job_cards = data["metaData"]["mosaicProviderJobCardsModel"]["results"]
    print(job_cards)
    # Parsing specific job details
    jobs = []
    for job in job_cards:
        # Use BeautifulSoup to remove any HTML tags in the description
        description_html = job.get("snippet", "")
        description = BeautifulSoup(description_html, "html.parser").get_text()

        job_info = {
            "title": job.get("title"),
            "company": job.get("company"),
            "location": job.get("formattedLocation") or "Not specified",
            "description": description,
            "rating": job.get("companyRating"),
            "review_count" : job.get("companyReviewCount") or "Not reviewed",
            "posted_date": job.get("formattedRelativeTime"),
            "job_link": "https://www.indeed.com" + job.get("link") if job.get("link") else None,
            "link" : job.get("thirdPartyApplyUrl"),
            "salary": job.get("salarySnippet", {}).get("text", "Not stated")  
        }
        jobs.append(job_info)
    
    return jobs

# Scraping configuration and execution
result = scrapfly.scrape(
    ScrapeConfig(
        url="https://www.indeed.com/jobs?q=" + role +"&l=" + location,
        asp=True,
    )
)

# Parsing results
parsed_jobs = parse_search_page(result.content)

# Saving to CSV file
if parsed_jobs:
    with open('indeed_jobs.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["title", "company", "location", "description", "rating", "review_count",  "posted_date", "job_link", "link", "salary"])
        writer.writeheader()
        writer.writerows(parsed_jobs)

    print("Data saved to indeed_jobs.csv")
else:
    print("No jobs data to save.")


Enter the role you are searching for: data engineer
Enter your prefered working location: remote


[{'appliedOrGreater': False, 'blobKey': 'SoB667M364q-_sURgZ0LbzkdCdPP', 'clickLoggingUrl': '/rc/clk?jk=af7fbc45c343b116&from=jasx&tk=1ibohc904j0pe85q&bb=pA8UrjmPyoia_s4rN9SfMO9BEk2aBTA1-k6VwSbcWY0XAYUKSG3WfjVdeiplpKxyMsEEjgKrEv3FYvlOPc0aF3yRfnUXdeW4bUa1zpUhkjVAaEzQxBux1PC5eQCg-eye&xkcb=SoB667M364q-_sURgZ0LbzkdCdPP&vjs=2', 'company': 'ScribeAmerica', 'companyBrandingAttributes': {'headerImageUrl': 'https://d2q79iu7y748jz.cloudfront.net/s/_headerimage/1960x400/9cd8632e569838a64e3486df1c9a5d96', 'logoUrl': 'https://d2q79iu7y748jz.cloudfront.net/s/_squarelogo/256x256/1392b6ba8b8797a581557555f3357a90', 'shownForBrandedJobPackage': False}, 'companyIdEncrypted': 'bbc337584760251c', 'companyOverviewLink': '/cmp/Scribe-America-9', 'companyOverviewLinkCampaignId': 'serp-linkcompanyname-content', 'companyRating': 3.1, 'companyReviewCount': 1898, 'companyReviewLink': '/cmp/Scribe-America-9/reviews', 'companyReviewLinkCampaignId': 'cmplinktst2', 'createDate': 1730407931000, 'd2iEnabled': False, 'di

In [113]:
import pandas as pd

df = pd.read_csv("indeed_jobs.csv")
df

,title,company,location,description,rating,review_count,posted_date,job_link,link,salary
0,Data Engineer,ScribeAmerica,Remote,\nMaintain and optimize data pipelines from Sa...,3.1,1898,2 days ago,https://www.indeed.com/rc/clk?jk=af7fbc45c343b...,http://www.indeed.com//applystart?jk=af7fbc45c...,"$80,000 - $85,000 a year"
1,Data Engineer,Endpoint Clinical,Remote,\nWork closely with data analysts and data sci...,3.1,15,2 days ago,https://www.indeed.com/rc/clk?jk=924f7c788f089...,http://www.indeed.com//applystart?jk=924f7c788...,"$100,000 - $140,000 a year"
2,Data Engineer,Tixr,Remote,\nDeep knowledge and experience with architect...,0.0,Not reviewed,30+ days ago,https://www.indeed.com/rc/clk?jk=dbba8c763d672...,http://www.indeed.com//applystart?jk=dbba8c763...,"$130,000 - $180,000 a year"
3,Senior Data Engineer,"Fabric Labs, Inc.",Remote,\nYou have experience extracting data from an ...,0.0,Not reviewed,30+ days ago,https://www.indeed.com/rc/clk?jk=3d465e6ca67b4...,http://www.indeed.com//applystart?jk=3d465e6ca...,Not stated
4,Sr. Data Engineer,Peer Supply,Remote,\nSet data architecture strategy for future pr...,0.0,Not reviewed,18 days ago,https://www.indeed.com/rc/clk?jk=73627954e36f6...,http://www.indeed.com//applystart?jk=73627954e...,Not stated
5,Senior Data Engineer - Kotlin,Tensure Consulting,Remote,"\nDesign, develop, and maintain scalable data ...",5.0,4,29 days ago,https://www.indeed.com/rc/clk?jk=a74bf8c7e22af...,http://www.indeed.com//applystart?jk=a74bf8c7e...,Not stated
6,Senior Data Engineer,Nike,"Beaverton, OR","\nAnticipate, identify and tackle issues conce...",4.1,12699,2 days ago,https://www.indeed.com/rc/clk?jk=b42fbc4e0c7d8...,http://www.indeed.com//applystart?jk=b42fbc4e0...,Not stated
7,Data Engineer II (Remote - US),Energy Solutions - USA,Remote,"\nDevelop and maintain data warehouses, optimi...",0.0,Not reviewed,2 days ago,https://www.indeed.com/rc/clk?jk=6f2b8b3c2985b...,http://www.indeed.com//applystart?jk=6f2b8b3c2...,"$110,000 - $130,000 a year"
8,Senior Data Engineer,Nike,"Beaverton, OR",\nDesign and implement data products and featu...,4.1,12699,2 days ago,https://www.indeed.com/rc/clk?jk=78ec992fa9982...,http://www.indeed.com//applystart?jk=78ec992fa...,Not stated
9,Data Engineer (Solution Lead),Mutual of Omaha,Remote,\nYou'll be responsible for defining and overs...,3.7,942,9 days ago,https://www.indeed.com/rc/clk?jk=faf062dc4aabf...,http://www.indeed.com//applystart?jk=faf062dc4...,"$110,000 - $145,000 a year"


In [ ]:
import re
import json
import csv
from scrapfly import ScrapflyClient, ScrapeConfig
from bs4 import BeautifulSoup

# Initialize Scrapfly with your API key
scrapfly = ScrapflyClient(key="Replace with your API key") #API key from scrapfly

role = input("Enter the role you are searching for:").strip() # Enter the job you are searching for
location = input("Enter your prefered working location:").strip() # Enter your prefered location such as Remote or your current location

def parse_search_page(html: str):
    # Extract JSON-like data for job listings from HTML
    data_match = re.findall(r'window.mosaic.providerData\["mosaic-provider-jobcards"\]=(\{.+?\});', html)
    
    if not data_match:
        print("No data found on page.")
        return None

    data = json.loads(data_match[0])
    job_cards = data["metaData"]["mosaicProviderJobCardsModel"]["results"]
    print(job_cards)
    # Parsing specific job details
    jobs = []
    for job in job_cards:
        # Use BeautifulSoup to remove any HTML tags in the description
        description_html = job.get("snippet", "")
        description = BeautifulSoup(description_html, "html.parser").get_text()

        job_info = {
            "title": job.get("title"),
            "company": job.get("company"),
            "location": job.get("formattedLocation") or "Not specified",
            "description": description,
            "rating": job.get("companyRating"),
            "review_count" : job.get("companyReviewCount") or "Not reviewed",
            "posted_date": job.get("formattedRelativeTime"),
            "job_link": "https://www.indeed.com" + job.get("link") if job.get("link") else None,
            "link" : job.get("thirdPartyApplyUrl"),
            "salary": job.get("salarySnippet", {}).get("text", "Not stated")  
        }
        jobs.append(job_info)
    
    return jobs

# Scraping configuration and execution
result = scrapfly.scrape(
    ScrapeConfig(
        url="https://www.indeed.com/jobs?q=" + role +"&l=" + location,
        asp=True,
    )
)

# Parsing results
parsed_jobs = parse_search_page(result.content)

# Saving to CSV file
if parsed_jobs:
    with open('indeed_jobs.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["title", "company", "location", "description", "rating", "review_count",  "posted_date", "job_link", "link", "salary"])
        writer.writeheader()
        writer.writerows(parsed_jobs)

    print("Data saved to indeed_jobs.csv")
else:
    print("No jobs data to save.")


Enter the role you are searching for: data scientist
Enter your prefered working location: remote


[{'appliedOrGreater': False, 'blobKey': 'SoBp67M364q2a2R7gJ0LbzkdCdPP', 'clickLoggingUrl': '/rc/clk?jk=3f42c150ca8155bb&from=jasx&tk=1ibohedcpgc2o800&bb=48JtnhKVQS8LkLzvQQi2yklj6e7lgHYBjzwA_C8i8tyUXMuZURz2xhrHqWknuC-KkTJUUGWNiqELii7LxhpyEHOYj8J0-03MOV-q2F-sbZ-1adYjpST5dE_K4FH3VCjQ&xkcb=SoBp67M364q2a2R7gJ0LbzkdCdPP&vjs=2', 'company': 'Anaconda, Inc.', 'companyRating': 0, 'companyReviewCount': 0, 'createDate': 1729695595000, 'd2iEnabled': False, 'displayTitle': 'Data Scientist, Product', 'dradisJob': False, 'employerAssistEnabled': False, 'employerResponsive': False, 'encryptedResultData': 'VwIPTVJ1cTn5AN7Q-tSqGRXGNe2wB2UYx73qSczFnGU', 'enhancedAttributesModel': {}, 'enticers': [], 'estimatedSalary': {'formattedRange': '$141K - $179K a year', 'max': 178695.34, 'min': 141124.72, 'type': 'YEARLY'}, 'expired': False, 'extractTrackingUrls': '', 'fccompanyId': -1, 'featuredCompanyAttributes': {}, 'featuredEmployer': False, 'featuredEmployerCandidate': False, 'feedId': 806159, 'formattedLocati

In [117]:
import pandas as pd

df = pd.read_csv("indeed_jobs.csv")
df

,title,company,location,description,rating,review_count,posted_date,job_link,link,salary
0,"Data Scientist, Product","Anaconda, Inc.",Remote,\nMentor junior data scientists and provide te...,0.0,Not reviewed,10 days ago,https://www.indeed.com/rc/clk?jk=3f42c150ca815...,http://www.indeed.com//applystart?jk=3f42c150c...,Not stated
1,Data Scientist,Southern New Hampshire University,Remote,\nExperience using data science to improve edu...,4.0,595,1 day ago,https://www.indeed.com/rc/clk?jk=0a7f0648d6261...,http://www.indeed.com//applystart?jk=0a7f0648d...,"$83,071 - $132,940 a year"
2,Data Scientist (L5) - Algo System Evaluation a...,Netflix,Remote,"\nYou’ll work across teams of data scientists,...",3.9,804,30+ days ago,https://www.indeed.com/rc/clk?jk=ba507ff631b5e...,http://www.indeed.com//applystart?jk=ba507ff63...,"$170,000 - $720,000 a year"
3,Data Scientist Mid to Senior Level,Fraym,Remote,\nContributing to the development of product o...,0.0,Not reviewed,2 days ago,https://www.indeed.com/rc/clk?jk=71b1401e3fabc...,http://www.indeed.com//applystart?jk=71b1401e3...,"$119,000 - $145,000 a year"
4,Data Scientist III,Hussmann Corporation,Remote,\nIntegrate with existing data storage solutio...,3.5,497,2 days ago,https://www.indeed.com/rc/clk?jk=df96c28337351...,http://www.indeed.com//applystart?jk=df96c2833...,"$85,800 - $165,000 a year"
5,Data Scientist (L5) - Ads (Measurement),Netflix,Remote,\nLeadership experience with large-scale data ...,3.9,804,9 days ago,https://www.indeed.com/rc/clk?jk=10a867d1671cd...,http://www.indeed.com//applystart?jk=10a867d16...,"$100,000 - $720,000 a year"
6,Senior Data Scientist,Development Gateway,Remote,"\nFor over 20 years, DG has researched data ec...",3.3,3,10 days ago,https://www.indeed.com/rc/clk?jk=551b849024b3f...,http://www.indeed.com//applystart?jk=551b84902...,$50 - $75 an hour
7,Marketing Data Scientist I,Nebraska Furniture Mart,Remote,\nUnravel: Verify the integrity of data used f...,3.6,775,3 days ago,https://www.indeed.com/rc/clk?jk=0aaaff418d38c...,http://www.indeed.com//applystart?jk=0aaaff418...,"$65,266 - $106,850 a year"
8,"Data Scientist, Hard goods",eBay,"San Jose, CA 95125",\nJoin a high-performing team of data scientis...,4.0,2509,3 days ago,https://www.indeed.com/rc/clk?jk=07f9a166ec3b8...,http://www.indeed.com//applystart?jk=07f9a166e...,"$135,200 - $205,700 a year"
9,Supervisory Data Scientist,US Technology Transformation Service,Remote,\nCoaches the USDC Fellows on determining appr...,0.0,Not reviewed,3 days ago,https://www.indeed.com/rc/clk?jk=4c5faf1a9d6a9...,http://www.indeed.com//applystart?jk=4c5faf1a9...,"$122,198 - $191,900 a year"
